In [1]:
import json
import requests
import numpy as np
import pandas as pd
import requests

In [2]:
credentials = json.loads(open('credentials.json').read())
token =  credentials['token']

In [3]:
headers = {'Authorization': 'token ' + token}
data = requests.get('https://api.github.com/user', headers=headers)
data = data.json()

In [4]:
url = data['repos_url']

In [5]:
url = data['repos_url']
page_no = 1
repos_data = []
while (True):
    response = requests.get(url, headers=headers)
    response = response.json()
    repos_data = repos_data + response
    repos_fetched = len(response)
    print("Total repositories fetched: {}".format(repos_fetched))
    if (repos_fetched == 30):
        page_no = page_no + 1
        url = data['repos_url'].encode("UTF-8") + '?page=' + str(page_no)
    else:
        break

Total repositories fetched: 15


In [6]:
#repos_data[5]

In [7]:
repos_information = []
for repo in repos_data:
    data = []
    data.append(repo['id'])
    data.append(repo['name'])
    data.append(repo['description'])
    data.append(repo['created_at'])
    data.append(repo['updated_at'])
    data.append(repo['owner']['login'])
    data.append(repo['watchers_count'])
    data.append(repo['url'])
    data.append(repo['commits_url'].split("{")[0])
    data.append(repo['languages_url'])
    repos_information.append(data)

In [16]:

repo_df_column = ['Id','Name','Description','Created on','Updated on','Owner','Watchers count', 
                    'Url','Commits url','Languages url']

In [18]:
repos_df = pd.DataFrame(repos_information,columns=repo_df_column)

In [10]:
for i in range(len(repos_df)):
    language_url = repos_df.loc[i,'Languages url']
    language_response = requests.get(language_url, headers=headers)
    language_response =  language_response.json()
    if language_response != {}:
        language=[]
        for key,value in language_response.items():
            language.append(key)
        language = ', '.join(language)
        repos_df.loc[i,'Language'] = language
    else:
        repos_df.loc[i,'Language'] = ""

In [11]:
repos_df.to_csv('repos_info.csv', index = False)

In [14]:
commit_information= []
for index, row in repos_df.iterrows():
    commit_url = row['Commits url']
    commit_response = requests.get(commit_url,headers=headers)
    commit_response = commit_response.json()
    for commit in commit_response:
        commit_data = []
        commit_data.append(repos_df.loc[i,"Id"])
        commit_data.append(commit['sha'])
        commit_data.append(commit['commit']['message'])
        commit_data.append(commit['commit']['author']['date'])
        commit_information.append(commit_data)
#commit_information

In [19]:
commit_df_column = ['Id','Sha','Commit Message','Commited on']

In [20]:
commit_df = pd.DataFrame(commit_information,columns=commit_df_column)

In [21]:
commit_df.to_csv('commit_info.csv',index = False)